# Ejercicio de análisis e interpretación de datos
## Curso Intersemestral “Modernización del manejo del cultivo del Arroz"
## Oscar Estrada - Alianza Bioversity CIAT


### 1. Cargar librerías

In [ ]:
# Librerias necesarias para la manipulacion de datos, exploracion y modelacion
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid") #whitegrid
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay

### 2. Cargar datos

In [ ]:
# Cargar datos desde GitHub
df = pd.read_csv('https://github.com/oestradavargas/Taller_UTolima/raw/main/arroz.csv', encoding='latin')

In [ ]:
# Ver tamaño de la base de datos (filas, columnas)
df.shape

In [ ]:
# Ver el encabezado de la base de datos
df.head()

In [ ]:
# Identificacion de variables numericas
var_numericas =  (df.drop('REND', axis=1)).select_dtypes('number').columns
var_numericas

In [ ]:
# Codificacion de variables dicotomicas SI/NO
#pd.set_option('future.no_silent_downcasting', True)
df = df.replace({'Si': 1, 'No': 0})

In [ ]:
# Ver cantidad de datos y tipo de variables
df.info()

### 3.1. Análisis exploratorio (numérico)

In [ ]:
# Identificar las variables con datos faltantes NA
df.isna().sum(0).sort_values(ascending=False).head(10)

In [ ]:
# Eliminar registros con datos faltantes
df = df.dropna()

In [ ]:
df.isna().sum(0).sort_values(ascending=False).head(10)

In [ ]:
# Resumen estadistico de las variables numericas
df.describe()

In [ ]:
# Descripcion de las variables categoricas
df.describe(include='object')

In [ ]:
# Exploracion de una variable categorica individualmente
df['TEXTURA'].value_counts()

In [ ]:
# Eliminar una variable
df = df.drop('TEXTURA', axis=1)

### 3.2 Análisis exploratorio (gráfico)

In [ ]:
# Histograma de la libreria Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='REND', bins=18, kde=True)
plt.title('HISTOGRAMA DE RENDIMIENTO', fontsize=16, fontweight='bold')
plt.xlabel('Rendimiento (ton/ha)', fontweight = 'bold')
plt.ylabel('Frecuencia', fontweight = 'bold')
plt.tight_layout()
plt.show()

In [ ]:
# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='REND')
plt.title('BOXPLOT DE RENDIMIENTO', fontsize=16, fontweight='bold')
plt.xlabel('(ton/ha)', fontweight = 'bold')
plt.tight_layout()
plt.show()

In [ ]:
# Boxplots de fertilizantes
fig, axes = plt.subplots(1, 3, figsize=(10, 6))
sns.boxplot(y='N_TOTAL', data=df, color = '#1f77b4', ax=axes[0])
axes[0].set_title('N TOTAL', fontweight = 'bold')
axes[0].set_ylabel('(kg/ha)', fontweight = 'bold')
sns.boxplot(y='P2O5_TOTAL', data=df, color = '#1f77b4', ax=axes[1])
axes[1].set_title('P2O5 TOTAL', fontweight = 'bold')
axes[1].set_ylabel('(kg/ha)', fontweight = 'bold')
sns.boxplot(y='K2O_TOTAL', data=df, color = '#1f77b4', ax=axes[2])
axes[2].set_title('K2O TOTAL', fontweight = 'bold')
axes[2].set_ylabel('(kg/ha)', fontweight = 'bold')
fig.suptitle('BOXPLOTS DE FERTILIZANTES QUIMICOS TOTALES', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Grafico de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, y='REND', x='N_TOTAL')
plt.title('N_TOTAL vs RENDIMIENTO', fontsize=16, fontweight='bold')
plt.xlabel('N_TOTAL (kg/ha)', fontweight = 'bold')
plt.ylabel('Rendimiento (ton/ha)', fontweight = 'bold')
plt.tight_layout()
plt.show()

In [ ]:
# Eliminar valores fuera de rango para los fertilizantes
df = df[(df['N_TOTAL'] >= 0) & (df['N_TOTAL'] <= 250*1.3)]
df = df[(df['P2O5_TOTAL'] >= 0) & (df['P2O5_TOTAL'] <= 100*1.3)]
data = df[(df['K2O_TOTAL'] >= 0) & (df['K2O_TOTAL'] <= 150*1.3)]

In [ ]:
# Grafico de frecuencias de variedades
orden = ['Otro', 'Fedearroz_67', 'Fedearroz_2020', 'Fedearroz_Ibis', 'Fedearroz_2000',
        'Fedearroz_68']

plt.figure(figsize=(10, 6))
ax = sns.countplot(x="VARIEDAD", data=df, order=orden, color = '#1f77b4')

# Agregar etiquetas de frecuencia en cada barra
for p in ax.patches:
    ax.annotate(f'{p.get_height()} ({p.get_height() / len(df) * 100:.2f}%)',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=12)

plt.title('DISTRIBUCION DE VARIEDADES', fontsize=16, fontweight='bold')
plt.xlabel('Variedad', fontweight = 'bold')
plt.ylabel('Frecuencia', fontweight = 'bold')
plt.tight_layout()
plt.show()

### 4. Modelacion (Random Forest)

In [ ]:
# Codificar las variables categoricas
df = pd.get_dummies(df, drop_first=True, dtype = int)
df = df.reset_index(drop=True)

In [ ]:
#### Definicion de X y Y
X, y = df.drop(columns="REND"), df["REND"]

In [ ]:
### Particionamiento de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 42)

In [ ]:
# Identificacion de las variables categoricas
var_categoricas = list(set(X_train.columns) - set(var_numericas))
var_categoricas = pd.Index(var_categoricas)
var_categoricas

In [ ]:
# Modelo RandomForest
modelo = RandomForestRegressor(random_state=4,
                               n_jobs=-1)

modelo.fit(X_train, y_train)

In [ ]:
# Metricas de evaluacion (R2)
train_score = modelo.score(X_train, y_train)*100
print(f'R2 de entrenamiento: {train_score:.3f} %')
test_score = modelo.score(X_test, y_test)*100
print(f'R2 de prueba: {test_score:.3f} %')

### 5.1 Interpretación: Gráfico de relevancia de variables

In [ ]:
# Grafico de relevancia de variables
n_perf=20

nombres_var = modelo.feature_names_in_
data_model = pd.concat([pd.Series(nombres_var),
                        pd.Series(modelo.feature_importances_ )], axis=1)
data_model.columns = ['Feature', 'Values']
data_model = data_model.sort_values('Values', ascending=False)
data_model = data_model.iloc[0:n_perf]

plt.figure(figsize=(10, 6))
sns.barplot(data=data_model, x='Values', y='Feature', color='#1f77b4')
plt.title('Relevancia de Variables', size=16, fontweight='bold')
plt.xticks(rotation=90)
plt.xlabel('Relevancia media', fontweight='bold')
plt.ylabel('Variable', fontweight='bold')
plt.tight_layout()
plt.show()

### 5.2 Interpretación: Gráfico de dependencia parcial

In [ ]:
# Grafico de dependencia parcial de las variables
variables = ['RAD_SOL_ACUM']

common_params = {
    "subsample": 0.999,
    "n_jobs": 2,
    "grid_resolution": 10,
    "random_state": 42
}

features_info = {
    # features of interest
    "features": variables,
    # type of partial dependence plot: average, individual, both
    "kind": "average",
    # information regarding categorical features
    "categorical_features": var_categoricas
}

display = PartialDependenceDisplay.from_estimator(
    modelo,
    X_train,
    **features_info,
    **common_params,
)